In [ ]:
###############################################
## Import necessary libraries and files
###############################################
import numpy as cp
try:
    import cupy
    if cupy.cuda.is_available():
        cp = cupy
except:
    pass
import matplotlib.pyplot as plt
import time
import sys
from class_rbm import class_RBM
from aux import binarize_data
from aux import get_dataset

In [ ]:
###############################################
## Control Panel
###############################################

# hyperparameters of the cRBM
model = ['mnist', 'hKH', 'lecun', 4e-2, 0.4, 100]

addrss = 'out/'
label = 'p1.5decay'
dataset = model[0]            #{'mnist','kmnist'} --dataset
KH = model[1]                 #{None, 'hKH', 'vKH', 'hvKH'} --KH Modulation approach
training = 'CD'               #{'CD', 'PCD'} --training algorithm of the cRBM
k = 1                         # number of Gibbs sampling steps
epochs = 500                  # number of epochs (S)
W_init = model[2]             #{'lecun', 'std'} --weight initialization
eps0 = model[3]               # KH learning rate
eps_d = True                  #{False, True} -- decays with a polynomial eps0*(1- t/epochs)^(3/2) 
delta = model[4]              # KH penalty term
seed = 1234                   # random seed
p = 2.0                       # KH Lebesgue power
l = 2                         # KH ranking parameter l >= 2
if W_init == 'std':
    R = 1.0                   # Lebesgue p norm radius 
else:
    R = 0.1                   # Lebesgue p norm radius
batch_size = 100              # batch size (B)
eta = 1e-1                    # learning rate of the cRBM
N = 784                       # number of visible units
M = model[5]                  # number of hidden units
n_cl = 10                     # number of classes (C)

###############################################
## Loading and preprocessing dataset
###############################################

train_data, valid_data, test_data, train_labels, valid_labels, test_labels = get_dataset(dataset=dataset)

train_data = np.concatenate((train_data,valid_data))
train_labels = np.concatenate((train_labels,valid_labels))

# data binarization
X_train = binarize_data(train_data)
X_test = binarize_data(test_data)


X_train = np.concatenate((X_train,train_labels),axis=1)
X_test = np.concatenate((X_test,test_labels),axis=1)

X_train = cp.array(X_train)
X_test = cp.array(X_test)

In [ ]:
start_time = time.time()
cp.random.seed(seed)
rbm = class_RBM(N=N,M=M,n_cl=n_cl,eta=eta,batch_size=batch_size,W_init=W_init)
rbm.train(X_train,X_test, epochs=epochs,incr=incr, k=k,training=training,KH=KH,R=R,l=l,delta=delta, p=p,eps0=eps0,
          label=label,addrss=addrss,save_checkpoints=False,track_learning=True,save_learn_funcs=True,save_params=True,
          plot_weights=False,eps_d=eps_d,dataset=dataset,seed=seed)
end_time = time.time()  # Record the end time for the entire training process
total_training_time = end_time - start_time

print('Total training time: %.2f seconds' % total_training_time)